In [90]:
import numpy as np
import re
files = ['100k.tsv', '85k.tsv']

In [98]:
urls, overlap, p = {}, {}, []
for file in files:
    with open(file,'r') as f:
        for l in f:
            l = l.strip().split('\t')
            if l[14] not in urls:
                urls[l[14]] = 0
            urls[l[14]] += 1
for file in files:
    with open(file,'r') as f:
        for l in f:
            l = l.strip().split('\t')
            if urls[l[14]] > 1:
                if l[14] not in overlap:
                    overlap[l[14]] = []
                overlap[l[14]].append(set(re.sub(' +', ' ',' '.join(l[16:]).strip()).split(' ')))

In [106]:
p = []
for url in overlap:
    p.append(len(overlap[url][0] - overlap[url][1])/len(overlap[url][0]))

In [107]:
p

[0.0,
 0.3333333333333333,
 0.3333333333333333,
 0.5,
 0.0,
 0.8571428571428571,
 0.6666666666666666,
 0.0,
 0.6666666666666666,
 0.0,
 0.25,
 0.5,
 0.4,
 0.625,
 0.7142857142857143,
 0.0,
 0.5,
 1.0,
 0.6,
 0.0,
 0.2,
 0.5,
 0.5,
 0.5714285714285714,
 0.25]

In [108]:
np.mean(p)

0.39871428571428574

In [31]:
set(overlap[url][0])

{'kp1', 'kp2', 'kp3'}